## Lab 8, Part 2:   Recurrent Neural Networks (RNN)  -- Extra Credit

When it comes to model sequential data such as sentences, documents and videos, etc, the state of the art approach is to use Recurrent neural network (RNN). At each timestep, RNN takes an element (such as a word) as input, combines with past information encoded as a vector (such as all information in the sentence before this timestep), generate a new vector encoding both current input and past information, then delivers it to next timestep.

For more details about LSTM (a very popular variant of RNN), please refer to http://colah.github.io/posts/2015-08-Understanding-LSTMs/ and here is a very good video explaining RNN: https://www.youtube.com/watch?v=WCUNPb-5EYI.

### Generating text with Long Short-Term Memory Networks

RNN can be used to generate text. For more information, please read: https://karpathy.github.io/2015/05/21/rnn-effectiveness/.

The following is an example script to generate text from Nietzsche's writings.

Note: 
- At least 10 epochs are required before the generated text
starts sounding coherent, but more is better.

- It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

- If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [1]:
#Import necessary libraries 
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
# from IPython.display import clear_output

Using TensorFlow backend.


In [2]:
#Get the data - available from amazon
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower() # make it all lowercase 
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

corpus length: 600893
total chars: 57


In [3]:
# Cut the text in semi-redundant sequences of maxlen characters
## Cut the text into a series of windows. 
## Each window is 40 characters
## The window moves 3 steps forward each step

maxlen = 40
step = 5
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

# Turn these sentances into one-hot encoded vectors
## For all words in the sentances, there is a one, else there is a zero in that index of the vector

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print('Done!')

nb sequences: 120171
Vectorization...
Done!


Now we have data to feed a model for text generation. Next  we build a LSTM model to fit the data. Using Keras this is only few lines of code!

In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print('Done!')

Build model...
Done!


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.ma.log(preds)
    preds = preds.filled(0)
    preds = preds / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # clear_output()
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % (epoch+1))

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.4, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

# Training
-  Each epoch takes up to 1 minute or so on a CPU (an epoch took 30 seconds for my PC)
-  Recall that training on at least 20 epochs will give intelligible results 
-  So you're gonna have to let that puppy run for a while (if ETA per epoch is bigger than 5 minutes on your machine, you can reduce the number of epochs)

In [6]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=10,
          callbacks=[print_callback])

Epoch 1/10
120171/120171 [==============================] - 140s 1ms/step - loss: 2.1049

----- Generating text after Epoch: 1
----- diversity: 0.2
----- Generating with seed: "nd thereby perhaps serve this age as its"
nd thereby perhaps serve this age as itself and the self--the world all action of the self-in the some have the comprection and all in the for the possent of the proved the some all the some have the sould and all sould all the self--the self--the consersing that the proves of the religines of the some have proved the self--and the sould the proved the some all the self--and in the sore all of the sould and all all the soul the self--an
----- diversity: 0.4
----- Generating with seed: "nd thereby perhaps serve this age as its"
nd thereby perhaps serve this age as itself the provering to live the loght and all to the ere and have i proved and refored the every some the possination of which is mas for in what the allomation of all is a seems of the deem and not in the for 

understand _ourselves_ we must in order with instance, the precisely of them. so the logical man of self-spirit of the soul of the more
of an internials them than and and soul leads and some pressing of many in order instance and spirits. shapeness which have been sympress which and the have because at the same of the gratically have it is thereby and so deeper be and and his deceived and law of some
precisely could that is you
----- diversity: 1.0
----- Generating with seed: "order to
understand _ourselves_ we must "
order to
understand _ourselves_ we must tostaicism, apput-we from the basued. in order notagexuen. a germand if e experience. reventionsar
excive bylace, is then ordan of moral age forstands than uniestedic of no wish germinaty.

 1. whull dasce. stepling the wholere, can "life has and owhourist,"--and look? certatir spirited, as be things and
then hisheliever is
develope of the indepense mance, enoughe alfoun deman and reals therefore 
Epoch 5/10
120171/120171 [=========

 basic to morality--and every philosophers of the same the contempt of the superal of the same the same a desire the same the same the same a soul of the same the supposed the spirit of the same that is the same the same all the higher sentiment of the same the same the same the world be all the supposite the world the strange the formant of the same the intellect that it is the more more the thing that is a spirit of the contempt of th
----- diversity: 0.4
----- Generating with seed: " basic to morality--and every philosophe"
 basic to morality--and every philosophers of his similard standation that we has the most heart of the laboriobs and conscience that they saint in the sentiment of the truth" and consequance of the same and the recourition, the condition to the extent of the supposed to the man and in the power of the same that which have been soul of an impation of the most have be a strange in the extent that it is nothing and the same and even when 
----- diversity: 0.5
-----

In [7]:
from numpy import *

## Load pre-trained model
Since it is time consuming to train this LSTM model with CPU for more epochs, we provided a pre-trained model which is trained on GPU for 100 epochs. Use the following code to check how coherency the model is.

It requires h5py packages, please install it to test the following code.

In [9]:
# build the model: a single LSTM
print('Load pre-trained model...')
from keras.models import load_model
model = load_model('shakespear200.h5')


def lstm_generate(seed, model):
    orig_seed = seed
    for diversity in [0.2, 0.3, 0.5, 1.0]:
        print('----- diversity:', diversity)
        seed = orig_seed
        generated = ''
        generated += seed
        print('----- Generating with seed: "' + seed + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(seed):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            seed = seed[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


seed = "from an anguish with which no other is t"
# seed = "thou art"
lstm_generate(seed, model)


Load pre-trained model...
----- diversity: 0.2
----- Generating with seed: "from an anguish with which no other is t"
from an anguish with which no other is the subjection of the more and such a possible the subjection of the sense of the present in the sense of the subjection to the sense of the soul of the subjection of the subjection of the states of the subjection of the subjection of the subjection of the sense of the states of the present the subjection of the sense of the sense of the more and the subjection of the more and something the state o
----- diversity: 0.3
----- Generating with seed: "from an anguish with which no other is t"
from an anguish with which no other is the consciousness and such man as the respect in the subjection of all the so the subjection of the spiritual common the so the subjection of the sense the subjection of the constitute of the contradiction of the sense of the state of the case have a pains and the possesses of the say to the state the self--a

### Exercise: try it to generate baby names
-  The following data set contains 8000 last names. You can download and process the name data set as follows:

```python
name_path = get_file('names.txt', origin='http://www.cs.cmu.edu/afs/cs/project/ai-repository/ai/areas/nlp/corpora/names/other/names.txt')
with io.open(name_path, encoding='utf-8') as f:
    text = f.read() # make it all lowercase 
    
text = text.split()
text = ', '.join(text)
```

Using the last name data set, answer the following tasks:

- Train a LSTM to generate the names.
- How long does it take to train? How coherent does it sound? 
- Can you train the LSTM, but for every epoch, shuffle the order of names before call model.fit()? How long does it take to train? Does it improve the coherency?

